# Import Dependencies

In [ ]:
# !pip install -U googlemaps

In [1]:
# Dependencies and Setup FOR API
import pandas as pd
import sqlite3 as sql 
import numpy as np
import requests
import json

import googlemaps
from api_keys import g_key

## Read and Store Data Science Jobs Salaries CSV file into DataFrame

In [2]:
# Read and Store linkedin_jobs_usa file into dataframe 
jobs_stats = "data/ds_salary.csv"
jobs_stats_df = pd.read_csv(jobs_stats, index_col=0)
jobs_stats_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


## Transform the Dataframe

In [3]:
# Check the null values (if any) in the df
jobs_stats_df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [4]:
# Check for duplicates
jobs_stats_df.duplicated().sum()

42

In [5]:
# Check the unique job titles in the df
print(jobs_stats_df['job_title'].nunique())
print(jobs_stats_df['job_title'].unique())

50
['Data Scientist' 'Machine Learning Scientist' 'Big Data Engineer'
 'Product Data Analyst' 'Machine Learning Engineer' 'Data Analyst'
 'Lead Data Scientist' 'Business Data Analyst' 'Lead Data Engineer'
 'Lead Data Analyst' 'Data Engineer' 'Data Science Consultant'
 'BI Data Analyst' 'Director of Data Science' 'Research Scientist'
 'Machine Learning Manager' 'Data Engineering Manager'
 'Machine Learning Infrastructure Engineer' 'ML Engineer' 'AI Scientist'
 'Computer Vision Engineer' 'Principal Data Scientist'
 'Data Science Manager' 'Head of Data' '3D Computer Vision Researcher'
 'Data Analytics Engineer' 'Applied Data Scientist'
 'Marketing Data Analyst' 'Cloud Data Engineer' 'Financial Data Analyst'
 'Computer Vision Software Engineer' 'Director of Data Engineering'
 'Data Science Engineer' 'Principal Data Engineer'
 'Machine Learning Developer' 'Applied Machine Learning Scientist'
 'Data Analytics Manager' 'Head of Data Science' 'Data Specialist'
 'Data Architect' 'Finance Data A

In [6]:
# Check the unique countries in the df
print(jobs_stats_df['company_location'].nunique())
print(jobs_stats_df['company_location'].unique())

50
['DE' 'JP' 'GB' 'HN' 'US' 'HU' 'NZ' 'FR' 'IN' 'PK' 'CN' 'GR' 'AE' 'NL'
 'MX' 'CA' 'AT' 'NG' 'ES' 'PT' 'DK' 'IT' 'HR' 'LU' 'PL' 'SG' 'RO' 'IQ'
 'BR' 'BE' 'UA' 'IL' 'RU' 'MT' 'CL' 'IR' 'CO' 'MD' 'KE' 'SI' 'CH' 'VN'
 'AS' 'TR' 'CZ' 'DZ' 'EE' 'MY' 'AU' 'IE']


In [7]:
# Rename the cells of remote_ratio column as 'onsite', 'hybrid' ,'remote' for plotting
jobs_stats_df.remote_ratio.replace([0,50,100], ['onsite', 'hybrid' ,'remote'],inplace = True)
jobs_stats_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,onsite,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,onsite,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,hybrid,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,onsite,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,hybrid,US,L


In [8]:
# Write the df to SQLite database
conn = sql.connect('data/jobstats_db.sqlite')
jobs_stats_df.to_sql('job_stats', conn, if_exists='replace', index=False)
pd.read_sql('select * from job_stats', conn)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,onsite,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,onsite,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,hybrid,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,onsite,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,hybrid,US,L
...,...,...,...,...,...,...,...,...,...,...,...
602,2022,SE,FT,Data Engineer,154000,USD,154000,US,remote,US,M
603,2022,SE,FT,Data Engineer,126000,USD,126000,US,remote,US,M
604,2022,SE,FT,Data Analyst,129000,USD,129000,US,onsite,US,M
605,2022,SE,FT,Data Analyst,150000,USD,150000,US,remote,US,M


## Read and Store USA jobs CSV file into DataFrame

In [9]:
# Read and Store linkedin_jobs_usa file into dataframe 
jobs_usa = "data/linkedin_jobs_usa.csv"
jobs_usa_df = pd.read_csv(jobs_usa)
jobs_usa_df.head()

,title,company,description,onsite_remote,salary,location,criteria,posted_date,link
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...


## Transform the DataFrame

### Add coordinates of the locations using Googlemaps

In [10]:
# gmaps = googlemaps.Client(key= g_key)

# jobs_usa_df['latitude'] = None
# jobs_usa_df['longitude'] = None

# for i in range(0, len(jobs_usa_df)):     
#     try:        
        
#         geocode_result = gmaps.geocode(jobs_usa_df.iat[i, 5])
#         lat= geocode_result[0]['geometry']['location']['lat']
#         lng= geocode_result[0]['geometry']['location']['lng']
#         jobs_usa_df.iat[i, jobs_usa_df.columns.get_loc('latitude')] = lat
#         jobs_usa_df.iat[i, jobs_usa_df.columns.get_loc('longitude')] = lng
        
#     except Exception as e:
#         print(e)
        
# jobs_usa_df  
       

### Save the updated dataframe as csv file - this file was used for further wrangling so that we are not makining the api call again

In [11]:
#  Save the new dataframe as CSV file 
# jobs_usa_df.to_csv('data/updated_jobs_usa.csv')

### Read and Store the updated CSV file into DataFrame

In [12]:
# Read and Store updated_jobs_usa file into dataframe 
updated_jobs_usa = "data/updated_jobs_usa.csv"
updated_jobs_usa_df = pd.read_csv(updated_jobs_usa, index_col=0)
updated_jobs_usa_df.head()

,title,company,description,onsite_remote,salary,location,criteria,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798


In [13]:
# Drop unwanted columns from the df
updated_jobs_usa_df = updated_jobs_usa_df.drop(columns=['criteria', 'description', 'salary'])


# Remane columns
updated_jobs_usa_df.rename(columns={'onsite_remote':'work_environment'}, inplace = True)

# Drop NaN values
updated_jobs_usa_df = updated_jobs_usa_df.dropna(subset=['latitude', 'longitude'])  

# Check type of columns 
updated_jobs_usa_df.dtypes


title                object
company              object
work_environment     object
location             object
posted_date          object
link                 object
latitude            float64
longitude           float64
dtype: object

In [14]:
# Write the df to SQLite database
conn = sql.connect('data/updated_jobs_usa_db.sqlite')
updated_jobs_usa_df.to_sql('updated_jobs_usa', conn, if_exists='replace', index=False)
pd.read_sql('select * from updated_jobs_usa', conn)

,title,company,work_environment,location,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,onsite,Buffalo-Niagara Falls Area,2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,onsite,"San Jose, CA",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,onsite,"Texas, United States",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,onsite,"Illinois, United States",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,onsite,"Chicago, IL",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798
...,...,...,...,...,...,...,...,...
2838,Junior Data Analyst,Iris Software Inc.,hybrid,"Texas, United States",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...,31.968599,-99.901813
2839,Data Analyst (SQL),Marwood Group,hybrid,New York City Metropolitan Area,2022-10-26,https://www.linkedin.com/jobs/view/data-analys...,40.712776,-74.005954
2840,Data Analyst,SmartSense by Digi,hybrid,"Mishawaka, IN",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,41.661993,-86.158616
2841,Data Analyst,Synergy Search,hybrid,Nashville Metropolitan Area,2022-11-14,https://www.linkedin.com/jobs/view/data-analys...,36.162646,-86.781497
